In [16]:
import csv
import os
import rdkit #version 2020.03.5
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit.ML.Descriptors import MoleculeDescriptors 
from padelpy import padeldescriptor #padel linux should be installed.

inputfile_smiles='Put_Prediction_File_Here/example.smi' # need to change name

#read smiles of molecules 
file = open(inputfile_smiles)
smiles = []
for line in file:
    if len(line.strip('\n'))>2:
        smiles.append(line.strip('\n'))
file.close()

#data for PaDEL
try:
    padeldescriptor(config='./descriptors.xml',maxruntime=10000,retainorder=True,standardizenitro=True,detectaromaticity=True)
    padeldescriptor(mol_dir=inputfile_smiles, d_file='./Put_Prediction_File_Here/PaDEL_FP.csv', fingerprints=True)
except:
    print('padel have problem')

#PaDEL re-order
file_Padel_1 = open('./Put_Prediction_File_Here/PaDEL_FP.csv','r')
mol_num=[];lines=[]
for line in file_Padel_1:
    mol_num.append(line.replace('"', '').split(',')[0].split('_')[-1])
    lines.append(line)
file_Padel_1.close()

file_Padel_2 = open('./Put_Prediction_File_Here/PaDEL_FP.csv','w')
for i in range(0,len(lines)):
    if i == 0 :
        file_Padel_2.write(lines[i])
    else:
        for j in range(1,len(mol_num)):
            if int(i) - int(mol_num[j]) ==0:
                file_Padel_2.write(lines[j])
                if len(lines[j].replace(',,','').split(',')) < len(lines[0].replace(',,','').split(',')):
                    print('line '+str(i)+'have problem!')
file_Padel_2.close()


#data for RDKit Des
mols=[]
for i in range(0,len(smiles)):
    q=i+1
    mols.append(Chem.MolFromSmiles(smiles[i]))   

smiles_list = [Chem.MolToSmiles(mol) for mol in mols]
descs = [desc_name[0] for desc_name in Descriptors._descList]
desc_calc = MoleculeDescriptors.MolecularDescriptorCalculator(descs)
descriptors = pd.DataFrame([desc_calc.CalcDescriptors(mol) for mol in mols])
descriptors.columns = descs
descriptors.index = smiles_list
index_list = list(map(str,list(range(len(mols)))))
y = pd.DataFrame(index_list)
y.index = smiles_list
y.columns = ["index"]
dataset = pd.concat([y, descriptors], axis=1)
dataset.to_csv('Put_Prediction_File_Here/Rdkit_Descriptor.csv')

#data for RdKit FP
with open('Put_Prediction_File_Here/FP_Morgan2.csv', 'w', newline='')as f:
    f_csv = csv.writer(f)
    for sm_num in range(0,len(smiles)):
        bits = []
        mol = Chem.MolFromSmiles(smiles[sm_num])
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
        bit = fp.ToBitString()
        for i in range(0, 2048):
            bits.append(bit[i])
        f.write(smiles[sm_num])
        f.write(',')
        f_csv.writerow(bits)
        bits.clear()
        
#Generate data for xtb
for i in range(0,len(smiles)):
    filename='test'+str(i+1)
    newFileBox = 'Put_Prediction_File_Here/xtb/'+str(filename)
    m = Chem.MolFromSmiles(smiles[i])
    m3d = Chem.AddHs(m)
    AllChem.EmbedMolecule(m3d, randomSeed=7)
    try:
        AllChem.MMFFOptimizeMolecule(m3d,maxIters=100)
    except:
        print(str(smiles[i])+'optimize fail')
    
    try:
        os.makedirs(newFileBox)
    except:
        print('already exist')
    path1=str(newFileBox)+ '/'+ str(filename) + '.xyz'
    f1 = open(path1, 'w')
    f1.write(Chem.MolToXYZBlock(m3d))
    f1.write("$write\noutput file=properties.out\n")
    f1.close()
    
#run xtb
for i in range(0,len(smiles)):
    filename='test'+str(i+1)
    if i == 0: 
        os.chdir(r'./Put_Prediction_File_Here/xtb/'+str(filename))
    else:
        os.chdir(r'../'+str(filename))
    
    os.system('xtb '+str(filename)+'.xyz --opt verytight > xtbout')
    os.system('rm  charges xtbopt.log xtbrestart xtbtopo.mol wbo')
os.chdir('../../../')

#Read xtb output
num = []
HOMO_fu4_Line=[]
HOMO_fu3_Line=[]
HOMO_fu2_Line=[]
HOMO_fu1_Line=[]
HOMO_Line=[]
LUMO_Line=[]
LUMO_zheng1_Line=[]
LUMO_zheng2_Line=[]
LUMO_zheng3_Line=[]
LUMO_zheng4_Line=[]
gap=[]
Feimi_level=[]
Energy=[]
dipole=[]

for i in range(0,len(smiles)):
    xtb_out_file = 'Put_Prediction_File_Here/xtb/test'+str(i+1)+ '/properties.out'
    if os.path.exists(xtb_out_file):
        result = open(xtb_out_file)
        num.append(i+1)
        lines=[]
        for line in result:
            lines.append(line.strip('\n'))
        result.close()
        for hang_line in range(0,len(lines)):
            if '(HOMO)' in lines[hang_line]:
                HOMO_fu4_Line.append(lines[hang_line-4].split(' ')[-1])
                HOMO_fu3_Line .append (lines[hang_line-3].split(' ')[-1])
                HOMO_fu2_Line .append (lines[hang_line-2].split(' ')[-1])
                HOMO_fu1_Line .append (lines[hang_line-1].split(' ')[-1])
                HOMO_Line .append (lines[hang_line].split(' ')[-2])
                LUMO_Line .append (lines[hang_line+1].split(' ')[-2])
                LUMO_zheng1_Line .append (lines[hang_line+2].split(' ')[-1])
                LUMO_zheng2_Line .append (lines[hang_line+3].split(' ')[-1])
                LUMO_zheng3_Line .append (lines[hang_line+4].split(' ')[-1])
                LUMO_zheng4_Line .append (lines[hang_line+5].split(' ')[-1])
                try:
                    gap.append (abs(float(lines[hang_line].split(' ')[-2])-float(lines[hang_line+1].split(' ')[-2])))
                except:
                    gap.append(float(0))
            elif 'Fermi-level' in lines[hang_line]:
                Feimi_level .append (lines[hang_line].split(' ')[-2])
            elif 'TOTAL ENERGY' in lines[hang_line]:
                Energy .append (lines[hang_line].split(' ')[-5])
            elif 'Debye' in lines[hang_line]:
                dipole  .append ( lines[hang_line+2].split(' ')[-1])

result_write = open('Put_Prediction_File_Here/SemiL_LRP.csv','w')

result_write.write('num,H-4,H-3,H-2,H-1,H,L,L+1,L+2,L+3,L+4,gap,FermiLevel,TotalEnergy,Dipole\n')

for i in range(0,len(num)):
    result_write.write(str(num[i]))
    result_write.write(','+str(HOMO_fu4_Line[i]))
    result_write.write(','+str(HOMO_fu3_Line[i]))
    result_write.write(','+str(HOMO_fu2_Line[i]))
    result_write.write(','+str(HOMO_fu1_Line[i]))
    result_write.write(','+str(HOMO_Line[i]))
    result_write.write(','+str(LUMO_Line[i]))
    result_write.write(','+str(LUMO_zheng1_Line[i]))
    result_write.write(','+str(LUMO_zheng2_Line[i]))
    result_write.write(','+str(LUMO_zheng3_Line[i]))
    result_write.write(','+str(LUMO_zheng4_Line[i]))
    result_write.write(','+str(gap[i]))
    result_write.write(','+str(Feimi_level[i]))
    result_write.write(','+str(Energy[i]))
    result_write.write(','+str(dipole[i]))
    result_write.write('\n')
result_write.close()

In [ ]:
import joblib #0.16.0
from joblib import dump, load
import csv
import os
import numpy as np
import lightgbm as lgb #3.1.1
import sklearn #0.23.2
from sklearn import preprocessing

#inputfile_smiles='Put_Prediction_File_Here/example.smi'

#read smiles of molecules 
file = open(inputfile_smiles)
smiles = []
for line in file:
    if len(line.strip('\n'))>2:
        smiles.append(line.strip('\n'))
file.close()


file = open(inputfile_smiles)
smiles = []
for line in file:
    smiles.append(line.strip('\n'))
file.close()
# Read Several Files, load as x1(des1), x2(des2), name.
def read_file_descriptor(file_path,n=1,startline=1):
    x_input = []
    jishu=0
    with open(file_path, 'r',encoding='utf-8') as f:
        f_csv = csv.reader(f)
        for row in f_csv:
            jishu+=1
            if jishu>startline:
                x_input.append(row[n:])     
    x_input = np.array(x_input)
    x_input=x_input.astype('float')
    return x_input
def read_file_name(file_path):
    name=[]
    file = open(file_path)
    for line in file:
        name.append(line.strip('\n'))
    file.close()
    return name

x_semi = read_file_descriptor(file_path='Put_Prediction_File_Here/SemiL_LRP.csv',n=1,startline=1)
scaler = load('wholedataset_Model/scaler.pkl')
x_semi = scaler.transform(x_semi)
x_des = read_file_descriptor(file_path='Put_Prediction_File_Here/Rdkit_Descriptor.csv',n=2,startline=1)
x_Padel = read_file_descriptor(file_path='Put_Prediction_File_Here/PaDEL_FP.csv',n=1,startline=1)
x_Padel_part = read_file_descriptor(file_path='Put_Prediction_File_Here/PaDEL_FP.csv',n=2049,startline=1)
x_Morgan2 = read_file_descriptor(file_path='Put_Prediction_File_Here/FP_Morgan2.csv',n=1,startline=0)

name = smiles
x1 = np.concatenate((x_semi,x_Padel_part,x_Morgan2),axis=1)[:,0:]
x2 = np.concatenate((x_semi,x_des,x_Padel,x_Morgan2),axis=1)[:,0:]

# Load trained model.
def load_model_predict(model_file,x_dataset):
    reg_layer1 = load(model_file)
    x_pre = reg_layer1.predict(x_dataset)
    return x_pre

x_pre_first_layer=[]
print('Predicton Begin')
x_pre_first_layer.append(load_model_predict('wholedataset_Model/svr_des1.pkl',x1))
x_pre_first_layer.append(load_model_predict('wholedataset_Model/krr_des1.pkl',x1))
x_pre_first_layer.append(load_model_predict('wholedataset_Model/Lasso_des1.pkl',x1))
x_pre_first_layer.append(load_model_predict('wholedataset_Model/lgb_des2.pkl',x2))
x_pre_first_layer.append(load_model_predict('wholedataset_Model/xgboost_des2.pkl',x2))
x_pre_first_layer.append(load_model_predict('wholedataset_Model/GBRT_des2.pkl',x2))
x_pre_first_layer.append(load_model_predict('wholedataset_Model/Lasso_des2.pkl',x2))
x_pre_first_layer.append(load_model_predict('wholedataset_Model/RF_des2.pkl',x2))
print('Prediction Finish')

x_pre_first_layer_T=list(zip(*x_pre_first_layer))
x_pre_first_layer_T = np.array(x_pre_first_layer_T)

reg_second_layer = load('wholedataset_Model/Lars_secondlayer.pkl')
x_pre_second_layer = reg_second_layer.predict(x_pre_first_layer_T)

for i in range(0,len(name)):
    print(round(x_pre_second_layer[i]))